Dieses Jupyter-Notebook dient dazu, Sie in die Lage zu versetzen, selbst einfache Analysen per Topic-Modeling durchzuführen. Vieles ist nicht klausurrelevant, aber für Ihre weitere quantitative und digitale Arbeit mit Texten sehr hilfreich.

Das Notebook zeigt Ihnen die (bzw. eine mögliche) technische Implementierung mit Python und weist auf einige Dinge hin, die Sie beim Modellieren beachten sollten. Ein wenig wird dabei auch die statistische Theorie (ohne Formeln) erklärt, aber der Zugang ist möglichst oberflächlich angedacht, da die technische Umsetzung für den Anfang schon fordernd genug ist. Ich biete auch Zusatzinformationen zu weiteren Paketen, die bei späteren, komplizierteren Datenanalysen nützlich sein können (```pandas``` u.a.). Gleichzeitig wird auf Fallstricke und den breiteren Hintergrund des Topic Modeling hingewiesen.

Sie müssen nicht alles in diesem Notebook lesen. Alle Absätze, die *kursiv* gedruckt sind, beschreiben vorbereitende Algorithmen, Schleifen usw., die nicht für das Topic Modeling selbst unbedingt benötigt werden, en detail. Sie können den Code auch einfach ausführen und sich diese Details nicht durchlesen. Das Lernziel ist schließlich, dass Sie ein Topic Model durchführen können. Schaden kann es allerdings auch nicht... Wenn Ihnen etwas schon bekannt ist (etwa die Anmerkungen zum NLP-Workflow), brauchen Sie das natürlich auch nicht lesen. Auch der Teil zur Latent Semantic Analysis ist nicht zwingend, da die Latent Dirichlet Allocation das wichtige Verfahren ist, aber wenn Sie es lesen, erhalten Sie ein breiteres Verständnis des Konzepts.

**Bedienungsanleitung**: Lesen Sie sich erst einmal das Notebook durch, insbesondere auch die Zusammenfassung. Versuchen Sie dann, zu experimentieren (die Aufgaben geben Ihnen dabei Vorschläge). Die abschließende Aufgabe ist für eigene Experimente gedacht, in denen Sie mit selbst ausgewählten Daten arbeiten. Wenn Sie weitere Informationen zu den Funktionen brauchen, googlen Sie diese am besten zusammen mit dem Modulnamen.

**Wie funktioniert ein Jupyter-Notebook?**
- Interaktive Arbeit, statt, dass das "Programm" insgesamt durchläuft, läuft der Code Codezelle für Codezelle bei Aktivierung in einer Pythoninstanz im Hintergrund durch. Das ermöglicht ein flexibles Experimentieren, gerade im Bereich Datenanalyse!
- Codezellen kann man einzeln ausführen: Hineinklicken und Strg+Enter
- Wenn eine Zelle noch nicht ausgeführt wurde, dessen Output von einer späteren Zelle benötigt wird, wird die spätere Zelle nicht funktionieren
- Text kann in den Codezellen und Textzellen beliebig verändert werden (Speichern mit Strg+s)
- Änderungen am gesamten Notebook speichert man ebenfalls mit Strg+s
- Neue Zellen können Sie in der Leiste ganz oben mit dem "+"-Zeichen einfügen und dann in dem Dropdown-Menü entweder als Code- oder Textzelle (Markdown) kennzeichnen
- Wenn Sie eine Textzellen (Markdown) anlegen, erscheint diese erst in "hübsch", wenn Sie Strg+Enter drücken
- Schreiben Sie ruhig Notizen in eigenen Textzellen (oder meinen)!
- Kopieren Sie ruhig Code in eigene Codezellen und probieren Sie Dinge aus!

# Teil 1

### Importe

Neben den sicherlich schon bekannten Modulen ```string```, ```re``` und ```nltk``` sind drei für die Verarbeitung von (Text)daten sehr wichtige Module zu nennen.

- **pandas** liefert flexible Datenstrukturen, die tabellenartig sind und die sich sql-artig manipulieren lassen. Dies ist ein für die Verarbeitung und spätere Modellierung komplexer Daten unerlässliches Modul und Industriestandard
- **numpy** erwei)tert die Standard-Python-Module um verbesserte Verarbeitungsmöglichkeiten für numerische Daten. Liefert auch verbesserte Listen (Series genannt), die etwa Grundlage für die Datenstrukturen in ```pandas``` sind
- **scikit learn** (sklearn) ist ein Modul, das verschiedenste Objekte und Methoden für Machine Learning bietet. Das Modul ist sehr gut logisch durchstrukturiert, weshalb Sie hier das meisten von dem finden werden, das Sie suchen (sofern Sie wissen, was Sie brauchen)

Für Topic Modeling im Speziellen
- **gensim** bietet Objekte und Funktionen für Topic-Modeling. Mit ```sklearn``` könnten Sie die meisten Dinge in ```gensim``` auch tun, aber gensim ist eine weniger schreibintensive Alternative
- **pyLDAvis** bietet einfache (und schöne) Visualisierungsmöglichkeiten der Ergebnisse von Topic-Modellen, die im Browser erscheinen (oder hier im Jupyter-Notebook)

In [1]:
## Mpdule für die Interaktion mit dem Betriebssystem (Dateien einlesen)
import os

## Module für die Stringverarbeitung
import string # Funktionen für strings
import re # Regular Expressions
import nltk # Natural Language Toolkit

## Module für vereinfachte Datenverarbeitung
import pandas as pd # Flexible, abfragbare Datenstrukturen: DataFrames (tabellenartig) 
import numpy as np # Für die Arbeit mit Zahlen (inklusive Methoden für Verfahren der linearen Algebra u.a.)

## Module (bzw. Funktionen) für Machine Learning
from sklearn.feature_extraction.text import CountVectorizer # Erstellung von häufigkeitsbasierten Document-Term-Matrizen
from sklearn.feature_extraction.text import TfidfVectorizer # Erstellung von tf-idf-basierten Document-Term-Matrizen

## Module für Topic-Modelling (und anderes, textbezogenes)
from gensim import corpora
from gensim.matutils import Sparse2Corpus
from gensim.models import LsiModel
from gensim.models import LdaModel

## Visualisierung (von LDA Topic-Modellen)
import pyLDAvis
import pyLDAvis.gensim # speziell nötig für den Output eines gensim LDA-Topic-Modells

### Daten, Daten einlesen und praktische Datenformate

Wir benutzen einen erst einmal einen Standard-Datensatz, den wir für ein interpretierbares Ergebnis nicht sehr groß verändern müssen. Es handelt sich um 20.000 Posts aus 20 Newsgroups (wir nehmen ein ca. 12.000 Posts großes Subset), welcher in den 1990ern gesammelt wurde und sehr oft für erste Tests von Algorithmen der Textklassifikation eingesetzt wird. Es ist Teil des Machine Learning Pakets ```sklearn``` und kann per ```import``` importiert und darauffolgend per Funktion heruntergeladen werden.

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

Das Objekt, das wir mit der Variable ```dataset``` assoziiert haben, hat mehrere Attribute, in denen die eigentlichen Daten liegen. Es gibt eigene Attribute unter anderem für die Dateinamen, die Textdaten selbst sowie das Label der Newsgroup, in der der Post entstanden ist.

Nun schreiben wir dies in einen ```pandas``` DataFrame. Diese kann man sich als einfache Tabellen vorstellen, die im Hintergrund aber zahlreiche praktische Abfragen und Funktionen erlaubt. Python ohne ```pandas``` bietet kein vergleichbares Format, so dass sich das Modul in der Datenverarbeitung als unverzichtbar herausgestellt hat. Daten für den Zeitraum einer Analyse so aufzubewahren ist flexibel und übersichtlicher, als sehr viele lange Listen und andere Konstrukte zu definieren.

*Mit ```pd.DataFrame()``` generieren wir einen Dataframe. Darin eröffnen wir in ```{}``` ein Dictionary, worin die Schlüssel in Anführungszeichen später die SPaltennamen sein werden. Die Werte dieser Schlüssel können alles mögliche sein, in diesem Fall aber die betreffenden Attribute aus dem 20newsgroups-Datenobjekt. Diese Listen müssen natürlich gleichlang sein, es soll ja alles in dieselbe Tabelle.*

*Daraufhin können wir mit ```data_all["filenames"]``` sehr einfach auf die Spalte zugreifen. Wenn man nichts weiter dahinterschreibt, wird einfach die Spalte ausgegeben. In diesem Fall ändern wir aber die Einträge, die noch den kompletten Dateipfad zu den Texten enthalten, auf die reinen Postnamen (mittels regulärem Ausdruck). Die Logik von ```apply``` und ```lambda```-Ausdrücken zu erlkären würde hier zu weit gehen, aber sie sind sehr gut offiziell dokumentiert. **Ändern Sie hier bitte heckelenme auf Ihren eigenen Usernamen.***

*Schlußendlich können wir uns den "Kopf" des Dataframes anzeigen lassen. Wenn man in ```head()``` eine Zahl einträgt, werden genauso viele Zeilen ausgegeben. Standardmäßig sind es fünf.*

In [7]:
data_all = pd.DataFrame({
        "filenames" : dataset.filenames,
        "texts" : dataset.data,
        "newsgroup": dataset.target
        })
data_all['filenames'] = data_all['filenames'].apply(lambda x: re.sub(re.escape('/home/heckelenme/scikit_learn_data/20news_home/20news-bydate-train/'), '', x))
data.head(10)

,filenames,newsgroup,texts
0,talk.politics.mideast/76141,17,Well i'm not sure about the story nad it did s...
1,alt.atheism/53281,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,talk.politics.mideast/76350,17,Although I realize that principle is not one o...
3,sci.crypt/15509,11,Notwithstanding all the legitimate fuss about ...
4,rec.sport.hockey/54242,10,"Well, I will have to change the scoring on my ..."
5,soc.religion.christian/20631,15,"\n \nI read somewhere, I think in Morton Smit..."
6,comp.sys.mac.hardware/51946,4,\nOk. I have a record that shows a IIsi with ...
7,talk.politics.mideast/75396,17,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...
8,sci.med/59227,13,Nobody is saying that you shouldn't be allowe...
9,sci.electronics/53549,12,\n I was wondering if anyone can shed any lig...


Einzelne Spalten können wir folgendermaßen ansprechen und dann wie eine übliche Listenindizierung in der Länge einschränken. Hier sprechen wir zunächst die Spalte ```filenames``` an und dann die ersten 10 Werte darin:

In [5]:
data_all['filenames'][0:10]

0     talk.politics.mideast/76141
1               alt.atheism/53281
2     talk.politics.mideast/76350
3                 sci.crypt/15509
4          rec.sport.hockey/54242
5    soc.religion.christian/20631
6     comp.sys.mac.hardware/51946
7     talk.politics.mideast/75396
8                   sci.med/59227
9           sci.electronics/53549
Name: filenames, dtype: object

Einzelne Zeilen kann man ansprechen, indem man ```.loc``` wie unten benutzt und dann erst die Zeile (bzw. eine Range der Form ```0:10```) und dann nach einem Komma die Spalte angibt (als Zahlenindex oder über den Spaltennamen). Gibt man bei Spalte oder Zeile ein ```:``` ein, werden alle Spalten oder Zeilen ausgegeben (je nachdem, ob rechts oder links vom Komma).

In [6]:
data_all.loc[0 , :]

filenames                          talk.politics.mideast/76141
newsgroup                                                   17
texts        Well i'm not sure about the story nad it did s...
Name: 0, dtype: object

Mit dem Attribut ```shape``` finden wir heraus, wieviele Zeilen und Spalten unser Dataframe hat.

In [7]:
data_all.shape

(11314, 3)

Das sind nicht wenige Zeilen und daher Texte. Wir wollen für unsere Demonstrationszwecke Texte haben, die sich möglichst voneinander unterscheiden und außerdem nicht so viele, damit die Berechnung schnell geht. Dafür haben wir die Newsgroups mit aufgenommen, die ja intern thematisch gut beschränkt sind. Nun können wir unseren Dataframe filtern, indem wir nur die Zeilen behalten, die bei ```newsgroup``` ein entsprechendes Label haben. Wenn wir die Newsgroups "soc.religion.christian", "talk.politics.mideast" und "rec.sport.hockey" wollen, sind das die Labels 15, 17 und 10 (schauen Sie ruhig oben bei den zehn ausgegebenen Zeilen aus dem DataFrame).

*Der nachfolgende Befehl fragt in jeder Zeile ab, ob der Wert unter ```newsgroup``` in einer Liste ```[15, 17, 10]```, also einer Liste der gewünschten Labels enthalten sein könnte. Dazu indizieren wir einfach, wie wir es schon gelernt haben und nehmen als Index diese Abfrage. Das Resultat einer solchen Abfrage ist eine Liste mit ```True, False, True, True usw.```. Wenn man eine solche Liste dann für die Indizierung der Zeilen nutzt, werden nur die Zeilen mit ```True``` widergegeben.*

In [8]:
data = data_all[ data_all["newsgroup"].isin([15, 17, 10]) ]
data.shape

(1763, 3)

### Was ist ein Thema?

Nun haben wir unsere Daten und wollen die Themen finden. Aber was sind Themen überhaupt? Auf einer theoretischen Ebene kann man dabei sehr ins Detail gehen, aber für die statistische Arbeit am Text müssen sich diese Ideen auch in etwas Zählbares übersetzen lassen. Dieses Mapping von theoretischen Konzepten zu formal in Zahlen ausdrückbaren Verhältnissen nennt man *Operationalisierung*. Und bei der Operationalisierung geht immer etwas verloren. Bei so etwas komplexen und oftmals von textexternen Dingen bestimmten wie einem Thema geht sogar eine ganze Menge verloren (namentlich der Kontext, Weltwissen des Lesers und so weiter).

Menschen überschätzen Ihre schließenden Fähigkeiten gern und vertrauen daher statistischen Ergebnissen, die plausibel erscheinen (Confirmation Bias). Je nichtlinearer etwas ist (je mehr Variablen bei einem Phänomen interagieren), desto höher ist aber der potenzielle und oftmals nicht akkurat quantifizierbare Fehler bei der verallgemeinernden Erklärung. Topic-Modelle sind reizbar, weil sie schnell zu etwas gut interpretierbarem führen (wie z.B. Stilometrie auch), das sich fast immer auf die Gedanken beziehen lässt, die man sich schon gemacht hat. 

Das Problem ist, dass es verschiedene Topic-Modelle gibt, die alle radikal unterschiedliche Annahmen und Voreinstellungsmöglichkeiten haben. Darüber hinaus gibt es schier unendlich viele Möglichkeiten der Textvorverarbeitung. Viele führen zu plausiblen Ergebnissen, sie können aber nicht alle stimmen. Das sollten Sie im Hinterkopf behalten, wenn Sie mit Topic-Modellen und statistischen Verfahren an sich arbeiten. Ich hoffe, ich konnte Sie zu einem erhöhten Skeptizismus alarmieren, denn davon kann es nie genug geben. 

In jedem Fall können solche Verfahren aber helfen, sich das Textmaterial systematisch zu erschließen und dies für die eigenen Überlegungen als *einen von mehreren Absprungspunkten* zu verwenden. Systematische Texterschließung und -klassifikation als rein pragmatisches Ziel ist auch der ursprüngliche Zweck dieser aus dem Information Retrieval stammenden Verfahren. Betrachten Sie den Output solcher Modelle einfach als statistisches Konstrukt, das nicht notwendigerweise mit Ihrem theoretischen Verständnis von Themen übereingeht (mit Sicherheit sogar nicht).

Wenn es um die Operationalisierung von Themen geht, kann man unterschiedliche auf Zählungen basierte Annahmen haben. Ein Thema könnte man unter anderem  auf folgende Weisen definieren (keine Angst, wenn Sie einige Begriffe nicht kennen):

1. **Wörter pro Dokument zählen und häufigste pro Dokument anschauen. Dahingehend sehr ähnliche Dokumente haben dasselbe Thema, dass sich durch die häufigsten Wörter beschreiben lässt**
2. **Wörter pro Dokument zählen und per Algebra latente (dahinterstehende) Variablen berechnen. Die Korrelationen dieser Variablen mit bestimmten Wörtern kennzeichnen den Inhalt dieser Themenvariablen. Dann kann man sich die Verteilungen dieser Variablen über die Texte anschauen**

    - **Variante 1 oder 2 mit Ausschluss bestimmter Wortarten, Berechnungen der Worthäufigkeiten über Sequenzen in den Dokumenten statt den ganzen Dokumenten, Berechnungen anderer Kennzahlen als Häufigkeit als Grundlage u.v.m.**


3. **Wörter pro Dokument zählen. Ein Thema ist einerseits eine Häufigkseitsverteilung von Wörtern und andererseits wird es dadurch bestimmt, wie sich diese Themenverteilungen über die Dokumente verteilen. Für eine vorher bestimmte Anzahl an Themen gibt es eine Optimalverteilung, die ein Algorithmus durch systematisches Ausprobieren (alternierendes Herumschieben von Wörtern zu Themencontainern und Themen zu Dokumenten) annähert. Überhäufige Wörter werden durch den Algorithmus automatisch heruntergewichtet, während für ein Dokument besondere Wörter hervorgehoben werden**

    - **Variante 3 mit den bereits erwähnten Vorverarbeitungen**

Variante 1 könnte eine Clusteranalyse sein, wie sie in anderer Form auch in der Stilometrie zum Einsatz kommt. Variante zwei ist ein Verfahren namens Principal Components Analysis, welches auch in der Stilometrie zum Einsatz kommt. Unter anderem Namen, **Latent Semantic Analysis / Indexing**, wird es als eines der ersten und weitverbreitesten Topic-Modelle eingesetzt (im Natural Language Processing, nicht den DH). Die Ergebnisse sind aber sehr abhängig von der Vorverarbeitung, namentlich, dass man primär Inhaltswörter zählt, u.a.. Es wurde als beliebtestes Verfahren von Variante 3 abgelöst, der **Latent Dirichlet Allocation**. Dieses Verfahren liefert auch ohne Vorverarbeitung gut interpretierbare Ergebnisse (wenn gut eingestellt). Tatsächlich wird aber auch hier oft viel im Vorfeld an den Texten gearbeitet.

Wir schauen uns unterschiedliche Vorverarbeitungsschritte an sowie die beiden Verfahren Latent Semantic Analysis und Latent Dirichlet Allocation, wobei letzteres in den DH am häufigsten benutzt wird. Zunächst aber ein kurzer Überblick über die üblichen Arbeitsschritte bei solchen Analysen.

### NLP-Workflow

Es hilft, sich zunächst den Workflow von Datenanalysen im allgemeinen vorzustellen. Die begrifflichkeiten in der Grafik sind nicht fachwissenschaftlich, sondern oberflächliche Beschreibungen:

![Wickham/Grolemund (2014) R for Data Science](https://d33wubrfki0l68.cloudfront.net/795c039ba2520455d833b4034befc8cf360a70ba/558a5/diagrams/data-science-explore.png)

Zunächst müssen Daten importiert werden und zwar in einer bestimmten Form. Das kann etwa als so ein DataFrame sein, aber es könnte auch eine Liste mit ```strings``` sein. 

Dann müssen wir die Daten säubern. Im Falle von Textdaten nennt man das auch *Textnormalisierung*. Es gibt verschiedene optionale Schritte, die zumeist auf Token hinauslaufen, also das, was man am Ende immer zählen will, zum Beispiel:

1. Vorbereitung:
    - Satzzeichen entfernen
    - Kleinschreibgung
    - Stoppwörter entfernen (besonders gängige oder andere, ungewollte)
    - Stemming / Lemmatisierung: Ersteres kappt naiv Wortendungen ab, letzteres führt auf Grundformen zurück
    - POS-Tagging: Wortartenerkennung (zum Beispiel zum Ausschluss von Wortarten)
    - Named Entity Recognition: Erkennung von Namen für Personen, Orte usw. (z.B. zwecks Ausschluss)
    - Syntaxparsing: Aufbau von Syntaxbäumen
2. Sequenzierung (optional): Aufteilen der Texte in Sequenzen, die dann statt der Gesamttexte als Dokumente gelten
3. Tokenisierung: Basierend auf dem bisherigen zerschneiden wir den Text in die Einheiten, die wir nachher pro Sequenz zählen möchten (Worte, N-Gramm, Sätze oder komplexere Strukturen (z.B. Phrasen)

Dies wird wieder gespeichert, idealerweise so, dass man es wiederfinden kann (z.B. in einer neuen Spalte des schon existierenden Dataframes), aber ohne die alten Texte zu überschreiben.

Nun haben wir alles Ungewollte entfernt und können unsere Texte so *transformieren*, dass wir die eigentliche Analyse beginnen können. Wie gesagt mussten wir erst einmal bestimmen, was gezählt werden soll und natürlich eine Tabelle erstellen. Wenn wir Worte zählen, würden wir für jedes Wort eine Spalte bestimmen und in der Zeile des Dokuments die Häufigkeit des Wortes abzeichnen. Das haut auch eine geometrische Interpretation: Nehmen wir an, es gäbe nur drei Worte. EIn Dokument könnte demnach unterschiedliche viele dieser Worte haben, also pro Wort eine Häufigkeit. Daher ist das Dokument ein Vektor (eine Zahlenreihe) mit drei Worthäufigkeiten. Geometrisch könnte man von 0 ausgehend nun einen Strich im dreidimensionalen Koordinatensystem ziehen. Man kann auch sagen, ein Dokument hat eine bestimmte Anzahl von Dimensionen / Variablen, die man sich als Richtungen vorstellt. Diese räumliche Interpretation kann man mathematisch nutzen, um sozusagen Abkürzungen zwischen den Worten zu finden, *latente Unterräume / Dimensionen*, die man dann Themen (oder andere Konstrukte, etwa stilistische Merkmale) nennen könne. Latent Semantic Analysis und Latent Dirichlet Allocation machen genau das. 

Wenn man so zählt und in eine Tabelle einträgt, geht natürlich sämtliche Information über die Struktur des Textes verloren, weshalb man auch **Bag of Words**-Modell sagt (dies gilt auch für stilometrische Analysen).

Wir müssen also nun in irgendeiner Form in ein Tabellenformat, eine Matrix, transformieren. Diese Transformation ist im NLP oft die zu einer **Document-Term-Matrix**. Eine Document-Term-Matrix ist eine Matrix, die Dokumentnamen als Zeilen hat und Worte / Token als Spalten: **Dokumente x Worte**

Sie ist die Grundlage (nach einiger Vorverarbeitung) für viele Verfahren im Natural Language Processing. Wichtig ist aber auch, was in den eigentlichen Zellen dieser Matrix steht. Zwei Beispiele:

- **(Relative) Häufigkeit** eines Wortes im betreffenden Dokument
- **TF-IDF**-Wert dieses Wortes für das Dokument

TF-IDF (Term-Frequency-Inverse-Document-Frequency) ist ein für Sie vermutlich neuer Begriff, aber er bezeichnet ein relativ intuitives Konzept. Die Metrik gewichtet die eigentlichen Häufigkeit eines Wortes anhand von dessen Gesamthäufigkeit im Corpus. Die Gewichtung wird so durchgeführt, dass für ein Dokument (im Sinne der Metrik) "wichtigere" Worte einen höheren Wert haben. Optimistisch kann man sagen, dass so Worte hervorgehoben werden, die den besonderen Themen eines Textes eher entsprechen.

Damit ist die Metrik unter anderem für Topic Modeling interessant und wird daher häufig für die Document-Term-Matrizen berechnet.

Haben wir unsere DTM, können wir zunächst beschreiben und visualisieren. Bei normalen Datenanalysen ist dieser erste Zugang zu den eigentlichen Daten der wichtigste Schritt, der uns viel Unnötiges später ersparen kann. Man erstellt Plots, errechnet Kennzahlen, um sich dem Verhältnis der verschiedenen Variablen zu nähern. Geht es um Häufigkeiten tausender Wörter, haben wir oft aber noch keine Vorstellung von irgendwelchen Zusammenhängen zwischen diesen Worthäufigkeiten. Von daher fällt dieser Schritt beim Topic-Modeling oft aus.

Wenn wir dann modellieren, mit dem eigentlichen Topic-Model (oder beliebigen anderen Verfahren), erhalten wir neue Erkenntnisse über die Daten. Ein streng wissenschaftliches Verfahren wäre es, eine Hypothese aufzustellen, möglicherweise in dem Modell zu falsizieren und dann war es das. Die eigentliche wissenschaftliche Praxis sieht aber eher wie der obige Zirkel aus: Man erkennt durch das Modellieren Eigenheiten der Daten, die man durch erneute Transformationen besser auffangen und in das Modell integrieren kann.

Irgendwann hat man dies soweit fortgeführt, dass die Ergebnisse anhand eines theoretischen oder statistischen Kriteriums valide erscheinen (oder Sie keine Lust mehr haben). Nun, kann man sie mittels Visualisierung oder anderweitiger Beschreibung herunterbrechen und im Seminar, auf einer Konferenz, vor Stakeholdern oder im Park kommunizieren.

In diesem Notebook durchlaufen wir alle Schritte, aber nicht die gesamte Textnormalisierung (nächstes Notebook). Die Funktionen für die Erstellung der Document-Term-Matrix nehmen uns einiges hiervon ab.

### Document-Term-Matrix erstellen

Mit ```gensim``` könnten Sie, nachdem Sie Ihre Texte tokenisiert haben, einfach mit dem Topic Modeling loslegen. Das Modul konvertiert beim Berechnen intern in eine Document-Term-Matrix. Da es aber sehr nützlich ist, dass auch "selbst" machen zu können und so eine DTM mal gesehen zu haben, machen wir das hier (die ```gensim```-Variante sehen Sie weiter unten).

Das Machine Learning - Modul sklearn bietet auch Datenformate für Textdaten. ```CountVectorizer()``` und ```TfidfVectorizer()``` erstellen aus den Texten solche Document-Term-Matrizen, wobei man zahlreichen Optionen festlegen kann (Satzzeichen werden immer entfernt):

- **analyzer**: Sollen Worte, Sätze oder etwas anderes (bestimmt per Regular Expression) gezählt werden?
- **stop_words**: Sollen Stoppwörter ausgeschlossen werden? Wenn ja, für welche Sprache?
- **lowercase**: Soll auf Kleinschriebung konvertiert werden?
- **ngram_range**: Sollen Ngramme berechnet werden? ( 1, 1) nimmt einfach nur ein Wort / Token
- **min_df**: In wie vielen Dokumenten muss ein Wort vorkommen, damit es aufgenommen wird?
- **max_df**: Selbsterklärend
- **max_features**: Wie viele der häufigsten Worte sollen aufgenommen werden?

Ihnen dürfte auffallen, dass dies den Einstellungen ähnelt, die Sie auch in ```stylo``` vornehmen können. Für Standardanwendungen reicht dies aus, doch wenn man speziellere Textnormalisierungen vornehmen will, muss man selbst Hand anlegen (im nächsten Notebook). 

Erst einmal instanziieren wir den Vectorizer nur mit den entsprechenden Parametern. Dann transformieren wir unsere Texte in der entsprechenden Spalte des Dataframes mit ```fit_transform```. Die resultierende DTM schreiben wir in eine Variable, hier X genannt. Die letzte Zeile schreibt mit der Methode ```get_feature_names()``` die Namen der Features, also hier die Tokennamen, in eine Liste. Diese brauchen wir später für das Topic Modeling mit ```gensim```.

In [9]:
# Umsetzung mit CountVectorizer()

vectorizer = CountVectorizer(
        analyzer = "word",
        stop_words = "english",
        lowercase = True,
        ngram_range = ( 1 , 1 ),
        min_df = 0,
        max_df = len(data_all["texts"]),
        max_features = 100 
        )
X = vectorizer.fit_transform(data_all["texts"])
wordlist_X = list(vectorizer.get_feature_names())

*Vom Umgang mit den Daten her ist es einfacher, in einen Dataframe umzuwandeln. Dazu benutzen wir wieder ```pd.DataFrame()```. Als Argumente müssen wir einmal die vektorisierten Daten geben (```X.toarray()```) sowie die Namen der Spalten bestimmen (die Feature Names). Die darauffolgende Zeile assoziiert eine neue Spalte ```filenames``` mit unseren Filenames. Diese Spalte wollen wir als Index für unsere Daten, sie soll nicht zu den Daten selbst gehören, was wir in der dritten Zeile bestimmen.*

In [11]:
# CountVectorizer-Objekt in DataFrame umwandeln (einfacher zu handhaben)

dataframe = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
dataframe["filenames"] = data_all.filenames
dataframe = dataframe.set_index("filenames")

In [12]:
dataframe[0:2] # die ersten zwei Zeilen der DTM

,00,10,12,14,15,16,20,25,a86,available,...,used,using,ve,want,way,windows,work,world,year,years
filenames,,,,,,,,,,,,,,,,,,,,,
talk.politics.mideast/76141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
alt.atheism/53281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


*Nun können wir von den Möglichkeiten der Dataframes profitieren und die Daten beliebig filtern und zusammenfassen. Wenn wir uns dafür interessieren, was die häufigsten Worte für den Beitrag ```talk.politics.mideast/76141``` sind, können wir nus das folgendermaßen ausgeben lassen.*

*Erst einmal wählen wir die erste Zeile des Dataframes aus. Wir transponieren sie, lassen also Spalten Zeilen sein, weil wir ja ein Ranking aufstellen wollen. Mit```sort_values()``` können wir dies tun, wobei wir die Spalte festlegen müssen, nach der geordnet werden soll. Es soll mit absteigender Reihenfolge geordnet werden. Mit ```head()``` können wir die Ausgabe dann beschränken.*

In [12]:
dataframe[0:1].transpose().sort_values(by="talk.politics.mideast/76141", ascending=False).head(5)

filenames,talk.politics.mideast/76141
look,1
try,1
did,1
sure,1
got,1


Das ist noch nicht sehr informativ, diese Worte sollten in den meisten Dokumenten ebenso häufig sein. Mit TF-IDF ist das anders. 

Das Vorgehen für ```TfidfVectorizer()``` ist exakt das gleiche:

In [13]:
# Umsetzung mit TfidVectorizer()

vectorizer_tfidf = TfidfVectorizer(
        analyzer = "word",
        stop_words = "english",
        lowercase = True,
        ngram_range = ( 1 , 1 ),
        min_df = 0,
        max_df = len(data_all["texts"]),
        max_features = 100, 
        sublinear_tf = True
        )
X_tfidf = vectorizer_tfidf.fit_transform(data_all["texts"])
wordlist_X_tfidf = vectorizer_tfidf.get_feature_names()

In [15]:
# TfidfVectorizer-Objekt in DataFrame umwandeln (einfacher zu handhaben)

dataframe_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names())
dataframe_tfidf["filenames"] = data_all.filenames
dataframe_tfidf = dataframe_tfidf.set_index("filenames")

In [15]:
dataframe_tfidf[0:1].transpose().sort_values(by="talk.politics.mideast/76141", ascending=False).head(5)

filenames,talk.politics.mideast/76141
government,0.369032
power,0.362757
world,0.354013
try,0.347334
look,0.337379


Nun haben wir zwei Konstrukte, auf denen wir Analysen rechnen können. Wie gesagt ist die Latent Dirichlet Allocation das für die DH relevantere Verfahren, aber um Ihnen die Variabilität von Themenkonzepten zu zeigen, berechnen wir auch einmal die Latent Semantic Analysis.

### Latent Semantic Analysis

```gensim``` muss zunächst jedem Wort eine ID geben, damit die internen Berechnungen funktionieren. Das geht mit dem ```corpora```-Untermodul und dessen Funktion ```Dictionary```. Wir tun dies für die Wortlisten von ```X``` und dessen mit TF-IDF berechnetem Pendant.

Dann können wir das ```LsiModel()``` berechnen, indem wir unsere DTM als Corpus angeben (tatsächlich wird eine Term-Document-Matrix erwartet, wir müssen also per ```transpose()``` umdrehen). Die Anzahl der Themen ist auf 15 begrenzt, es könnten aber mehr oder weniger sein. Die Wort-IDs stellen wir auf die gerade erstellten Dictionaries ein.

In [16]:
# Latent Semantic Analysis (Principal Components Analysis)

dictionary_X = corpora.Dictionary([wordlist_X])
dictionary_X_tfidf = corpora.Dictionary([wordlist_X_tfidf])

lsa = LsiModel(corpus=X.transpose(), num_topics=15, id2word=dictionary_X)

Wenn Sie gensims hauseigene Funktionen statt ```CountVectorizer()``` oder ```TfidfVectorizer()``` nehmen, sähe da so aus (ist für Latent Dirichlet Allocation identisch). Angenommen, text_data ist eine Liste mit den tokenisierten Texten (also eine list of lists):

- ```dictionary = corpora.Dictionary(text_data)```
- ```corpus = [dictionary.doc2bow(text) for text in text_data]```

```doc2bow()``` konvertiert in das Bag of Words Format. Die ```gensim```-Implementierung ist eine Liste an Zwei-Tupeln, daher ```(zahl, andere_zahl)```, wobei die erste Zahl die ID des Wortes ist und die zweite die Anzahl im Dokument. ```doc2bow()``` funktioniert immer nur für ein Dokument, daher müssen Sie wie im Code über diesem Absatz wieder eine list of lists bauen, die dann den Corpus darstellt.

- ```lsa = LsiModel(corpus=corpus, num_topics=15, id2word=dictionary_X)```

Anmerkung: Hier haben Sie nicht so viele Möglichkeiten, Ihre DTM (komfortabel) anzupassen, wie mit CountVectorizer() und Co.

Das Modell ist nun berechnet und wir können uns mit ```print_topics()``` die Themen anschauen. Der Output ist eine Gleichung, die man folgendermaßen interpretiert (selbiges gilt für LDA): Der Wert, den ein Thema für ein Dokument hat, setzt sich aus der Summe der Häufigkeiten der Wörter multipliziert mit deren jeweiliger Gewichtung zusammen. Die Werte vor den Wörtern zeigen also an, ob ein Thema bei einem Wort eher auftritt oder eher nicht.

In [17]:
lsa.print_topics(num_words=15)[0:3] # zeigt erste drei Themen, mit 15 Wörtern jeweils

[(0,
  '0.997*"ax" + 0.072*"max" + 0.016*"g9v" + 0.012*"b8f" + 0.010*"a86" + 0.001*"mr" + 0.001*"14" + 0.001*"25" + 0.000*"12" + 0.000*"ll" + 0.000*"15" + 0.000*"10" + 0.000*"16" + 0.000*"20" + 0.000*"new"'),
 (1,
  '0.782*"g9v" + 0.463*"b8f" + 0.411*"a86" + 0.047*"14" + 0.042*"mr" + -0.022*"max" + -0.021*"ax" + 0.019*"25" + 0.006*"10" + 0.004*"file" + 0.004*"15" + 0.003*"20" + 0.003*"know" + 0.003*"don" + 0.003*"16"'),
 (2,
  '0.472*"file" + 0.283*"edu" + 0.217*"use" + 0.197*"people" + 0.184*"don" + 0.164*"know" + 0.162*"available" + 0.161*"program" + 0.155*"com" + 0.153*"mr" + 0.146*"information" + 0.144*"like" + 0.140*"time" + 0.138*"new" + 0.130*"said"')]

Leider ist der Output nahezu unlesbar und sehr unübersichtlich. Für die LDA gibt es ja glücklicherweise ```pyLDAvis``` zur Visualisierung, aber damit Sie hier nicht ganz hilflos sind, habe ich es mal "von Hand" etwas übersichtlicher gestaltet.

In [18]:
topics = lsa.print_topics(num_words=15)
for topic in topics:
    combos = topic[1].split(" + ")
    weights = [float(combo.split("*")[0]) for combo in combos]
    terms = [combo.split("*")[1] for combo in combos]
    print("\nTopic " + str(topic[0]) + ":\n")
    print(pd.DataFrame({
        "weights": weights,
        "terms": terms
    }).sort_values(by="weights", ascending=False))


Topic 0:

    terms  weights
0    "ax"    0.997
1   "max"    0.072
2   "g9v"    0.016
3   "b8f"    0.012
4   "a86"    0.010
5    "mr"    0.001
6    "14"    0.001
7    "25"    0.001
8    "12"    0.000
9    "ll"    0.000
10   "15"    0.000
11   "10"    0.000
12   "16"    0.000
13   "20"    0.000
14  "new"    0.000

Topic 1:

     terms  weights
0    "g9v"    0.782
1    "b8f"    0.463
2    "a86"    0.411
3     "14"    0.047
4     "mr"    0.042
7     "25"    0.019
8     "10"    0.006
9   "file"    0.004
10    "15"    0.004
11    "20"    0.003
12  "know"    0.003
13   "don"    0.003
14    "16"    0.003
6     "ax"   -0.021
5    "max"   -0.022

Topic 2:

            terms  weights
0          "file"    0.472
1           "edu"    0.283
2           "use"    0.217
3        "people"    0.197
4           "don"    0.184
5          "know"    0.164
6     "available"    0.162
7       "program"    0.161
8           "com"    0.155
9            "mr"    0.153
10  "information"    0.146
11         "like"  

Wie Sie sehen können, ist eine gewisse Logik bei ein paar der "Themen" erkennbar. Leider sind noch immer relativ viele "obskure" Begriffe und Zahlen dabei. Diese können wir bei ```CountVectorizer()``` im Vorhinein ausschließen, indem wir ```max_features``` und ```min_df / max_df``` anpassen oder etwa eine eigene Stoppwortliste an ```stopwords``` geben. Wirklich sicher los werden wir die Störbegriffe aber nur durch ein paar weitere NLP-Vorverarbeitungsschritte (nächstes Notebook). Gewisse Abhilfe schafft aber auch schon die Nutzung der TF-IDF-DTM, da in dieser unwichtige Begriffe heruntergewichtet werden.

#### Aufgabe 1:

- Ändern Sie ```X.transpose()``` in ```LsiModel()``` zu ```X_tfidf.transpose()``` und ```dictionary_X``` zu ```dictionary_X_tfidf```
- Aktivieren Sie die Zelle noch einmal und lassen sich die Themen noch einmal ausgeben

### Latent Dirichlet Allocation

Bei der Latent Dirichlet Allocation ist die Vorbearbeitung mit TF-IDF nicht immer nötig, da das Verfahren automatisch eine optimale Wortgewichtung sucht. Die Funktion ist ```LdaModel()```. Angeben müssen Sie wieder die Anzahl der Themen, den Dictionary und den Corpus. Es gibt noch einige Anpassungsmöglichkeiten mehr, die aber den Rahmen dieses Notebooks sprengen würden. 

Für die LDA müssen Sie diesen allerdings noch in das spezielle Corpusformat von ```gensim``` umwandeln, mit ```Sparse2Corpus()```.

(Die Fehlermeldungen können Sie ignorieren)

In [21]:
# Latent Dirichlet Allocation

dictionary_X = corpora.Dictionary([wordlist_X])
dictionary_X_tfidf = corpora.Dictionary([wordlist_X_tfidf])

lda = LdaModel(
        corpus = Sparse2Corpus(X_tfidf.transpose()),
        num_topics = 8,
        id2word = dictionary_X_tfidf
        )

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

## Visualisierung

Das Modul ```pyLDAvis``` ermöglicht übersichtliche, interaktive Visualisierungen von Latent Dirichlet Allocation - Modellen. Ohne weitere Umschweife lassen wir so eine Visualisierung einmal durchlaufen.

Dazu müssen Sie das Modell erst einmal vorbereiten. Die Methode ```pyLDAvis.gensim.prepare()``` ermöglicht dies. Angeben müssen Sie 1. das Modellobjekt, 2. den Corpus / die DTM und 3. den zugehörigen Dictionary.

In [22]:
viz = pyLDAvis.gensim.prepare(lda, Sparse2Corpus(X_tfidf.transpose()), dictionary_X_tfidf)

In [23]:
# pyLDAvis.show(test) # dies müssen Sie benutzen, wenn Sie die Visualisierung in einem anderen Browserfenster erscheinen lassen wollen
pyLDAvis.display(viz)

Links sehen Sie die Ähnlichkeitsbeziehungen der Themen zueinander, wobei die Größe der Kreise den Anteil der Themen am Gesamtcorpus ausmacht. Sie können Sie anklicken, dann erscheinen rechts die häufigsten Begriffe dieses Themas. Rot sind dabei die Häufigkeiten innerhalb des Themas und blau die Häufigkeiten im Gesamtcorpus. Wie Sie sehen, gibt es auch hier einiges an "Rauschen", wobei sich einige der Themen durchaus den ausgewählten Foren zuordnen lassen könnten. Andere aber erscheinen eher wie grammatische Restkategorien.

Also scheint die LDA ohne Vorverarbeitung auch nicht viel besser zu performen, als die LSA. Das liegt daran, dass man bei der LDA nicht unbedingt die Texte anders normalisieren muss, sondern die Parameter des Algorithmus einstellen sollte. Der einfachste ist hierbei die Themenanzahl und weitere werden im nächsten Notebook besprochen. 

#### Aufgabe 2

- Ändern Sie die Anzahl der Themen und schauen Sie, wie sich das Ergebnis ändert
- Versuchen Sie es auch mit der TF-IDF-DTM (Einstellung funktioniert so wie in Aufgabe 1): Verbessert sich etwas?
- Ändern Sie die Einstellungen bei ```CountVectorizer()``` oder ```TfidfVectorizer()``` (zum Beispiel mehr oder weniger Worte mit ```max_df``` oder Stoppwortparameter weglassen). Führt hiervon etwas zu einem besseren Ergebnis?
    - Anmerkung: Nach so einer Änderung müssen Sie natürlich die Zellen nochmal aktivieren, inklusive aller, die sich auf diese beziehen
    
#### Aufgabe 3

- Führen Sie dieselben Experimente für die LSA durch

### Zusammenfassung

Das Einlesen des Corpus kann variieren, die Nachbearbeitung auch (im nächsten Notebook sehen Sie weitere Varianten), aber es gibt ein paar Funktionen, die Sie in den meisten Fällen gebrauchen. Dies hier sind die Funktionen:

**Document-Term-Matrix**:

- mit sklearn:
    - ```sklearn.feature_extraction.text.CountVectorizer()```
    - ```sklearn.feature_extraction.text.TfidfVectorizer()```
- mit gensim (alles passiert in nur einem Modul, aber etwas weniger customizable und nicht anderweitig nutzbar):
    - ```gensim.corpora.Dictionary() für die Erstellung des Wort-ID-Mappings```
    - das resultierende Dictionary-Objekt kann dann mit ```dictionary_objekt.doc2bow(text_als_tokenliste)``` die Textdaten in das nötige Format
    - gensim konvertiert das resultierende Objekt dann intern bei der Modellberechnung zu einer DTM

**Latent Dirichlet Allocation**:

- ```gensim.models.LdaModel()```
- wenn DTM mit sklearn erstellt wurde:
    - ```gensim.models.LdaModel(dtm_objekt.transpose(), num_topics=zahl, id2word=Dictionary_objekt)```
- wenn mit gensim-corpus:
    - ```gensim.models.LdaModel(corpus, num_topics=zahl, id2word=Dictionary_objekt)```

**Visualisierung**:

- Vorbereiten, wenn DTM mit sklearn:
    - ```pyLDAvis.gensim.prepare(Lda_objekt, Sparse2Corpus(dtm_objekt.transpose()), Dictionary_objekt)```
- Vorbereiten, wenn nur gensim vorbereitet:
    - ```pyLDAvis.gensim.prepare(Lda_objekt, corpus_objekt, Dictionary_objekt)```
- Darstellen in eigenem Browserfenster/-tab:
    - ```pyLDAvis.show(vorbereitetes_objekt)```
- Darstellen in einem Jupyter-Notebook:
    - ```pyLDAvis.display(vorbereitetes_objekt)```

# Teil 2

Wie es Ihnen vielleicht aufgefallen ist, muss man schon relativ stark an den Spezifikationen von einem Topic Model drehen, um ein subjektiv logisch erscheinendes Ergebnis zu erhalten. Selbst die eine zeitlang als state-of-the-art geltende Latent Dirichlet Allocation muss entweder entsprechend eingestellt werden (sehen Sie weiter unten) oder die Texte müssen weiter vorbereitet werden, als es mit den Standardfunktionen aus ```gensim``` und ```sklearn``` möglich ist (sehen Sie jetzt). Oder beides (das können Sie dann probieren).

Nachfolgende importieren wir alle Module und laden wir die Texte erneut, damit Sie nicht alle Zellen hierüber noch einmal ausführen müssen.

In [43]:
## Mpdule für die Interaktion mit dem Betriebssystem (Dateien einlesen)
import os

## Module für die Stringverarbeitung
import string # Funktionen für strings
import re # Regular Expressions
import nltk # Natural Language Toolkit

## Module für vereinfachte Datenverarbeitung
import pandas as pd # Flexible, abfragbare Datenstrukturen: DataFrames (tabellenartig) 
import numpy as np # Für die Arbeit mit Zahlen (inklusive Methoden für Verfahren der linearen Algebra u.a.)

## Module (bzw. Funktionen) für Machine Learning
from sklearn.feature_extraction.text import CountVectorizer # Erstellung von häufigkeitsbasierten Document-Term-Matrizen
from sklearn.feature_extraction.text import TfidfVectorizer # Erstellung von tf-idf-basierten Document-Term-Matrizen

## Module für Topic-Modelling (und anderes, textbezogenes)
from gensim import corpora
from gensim.matutils import Sparse2Corpus
from gensim.models import LsiModel
from gensim.models import LdaModel

## Visualisierung (von LDA Topic-Modellen)
import pyLDAvis
import pyLDAvis.gensim # speziell nötig für den Output eines gensim LDA-Topic-Modells

from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

data_all = pd.DataFrame({
        "filenames" : dataset.filenames,
        "texts" : dataset.data,
        "newsgroup": dataset.target
        })
# in der nachfolgenden Zeile heckelenme durch Ihren Nutzernamen ersetzen
data_all['filenames'] = data_all['filenames'].apply(lambda x: re.sub(re.escape('/home/heckelenme/scikit_learn_data/20news_home/20news-bydate-train/'), '', x))
data = data_all[ data_all["newsgroup"].isin([15, 17, 10]) ]
data = data.reset_index().drop("index", axis=1)

### Textnormalisierung

Die Literatur zum Topic Modeling ist sich uneinig darüber, inwieweit man Texte vor dem Topic Modeling vorbearbeiten muss. Logische Annahmen sind, dass Stoppworte grundsätzlich entfernt werden sollten und dass Substantive tendenziell mehr thematische Informationen bieten, als andere Wortarten. Letztendlich kommt es aber auch auf das Verfahren an, das benutzt werden soll. Während im Mainstream inzwischen kaum noch gebräuchliche Verfarhren wie die Laten Semnatic Analysis durchaus eine Textnormalisierung benötigen, ist das für das momentan noch gebräuchliche Verfahren Latent Dirichlet Allocation nicht unbedingt der Fall (Einstellung der Modellparameter kann schon Abhilfe bringen). 

Benchmark-Tests zeigen, dass das Entfernen von Stoppwörtern sowie Nicht-Substantiven keine nennenswerte Verbesserung des Ergebnisses bringt - der Latent Dirichlet Allocation - Algorithmus gewichtet überhäufige Wörter herunter und spezifische herauf (insofern er gut eingestellt ist). Das kommt aber auch auf die Textbasis an und Sie können sich denken, dass es gerade bei sehr spezifischen Textsorten wie etwa literarischer Prosa schwierig werden kann.

Was wir hier tun werden (Stoppworte können wir später mit ```CountVectorizer()``` noch entfernen:

- **Erkennung und Entfernung von Named Entities**(daher Figuren-, Ortsnamen u.a.)
- **Zeichensetzung entfernen** (tut etwa CountVectorizer() auch, wir tun es nur zur Sicherheit für den nächsten Schritt
- **Part-of-Speech-Tagging und Ausschluss von Nicht-Substantiven**
    
Theoretisch würde es auch helfen, zu lemmatisieren, daher die Wort-Token auf Ihre Grundform zurückzuführen. Dies ist aber wiederum zeitintensiv und hier kann der LDA-Algorithmus tatsächlich relativ gut differenzieren.

Als erstes laden wir dazu schwereres Geschütz, das Modul ```spacy```. Das Modul implementiert gängige und oft zeitintensive Algorithmen des NLP, die mit neuronalen Netzen und großen Daten trainiert wurden, in ```Cython```. Letzteres ist einfach gesagt eine Python-Version, die ```C``` unter der Haube hat und darum um ein Vielfaches schneller ist. Mit ```spacy``` kann man vieles tun, auch POS-Tagging, aber wir benutzen es nur für die Named Entity Recognition. Für das POS-Tagging benutzen wir ```nltk```, weil Sie das Modul schonmal gesehen haben dürften und die Syntax gut verständlich ist.

**Führen Sie die nachfolgenden beiden Codezellen NICHT aus, da sie auch auf dem Compute-Server Zeit in Anspruch nehmen (ca. 5 Minuten). Wenn Sie sie aus Versehen ausführen (nicht schlimm), klicken Sie in die betreffende Zelle und klicken Sie oben in der Leiste auf das quadratische "Stopp"-Symbol.**

*Wir importieren zunächst ```spacy``` per ```import```. Im nächsten Schritt müssen wir noch ein neuronales Netz für die englische Sprache laden und es an einen beliebigen Variablennamen binden (```nlp```). Den Variablennamen können wir nun wie eine Funktion benutzen und Texte damit bearbeiten. Bei der Anwendung kommt ein durchstrukturiertes Objekt heraus, aus dem Sie etwa die POS-Tags, Named Entities und einiges mehr herausbekommen können.*

In [24]:
import spacy
nlp = spacy.load('en_core_web_sm')

*Nun wollen wir die verschiedenen Aufgaben in einer Schleife erledigen. Zunächst erstellen wir eine leere Liste ```new_texts```, in die die bearbeiteten Textversionen abgelegt werden. Dann gehen wir in der DataFrame-Spalte ```texts``` durch jeden Eintrag, wobei der Eintrag an der Reihe im Rahmen der Schleife immer ```text``` heißen wird. Die Schleife ist so strukturiert, dass 1.) ```spacy``` alle NLP-Aufgaben erledigt, 2.) Named Entities ausgeschlossen werden, 3.) Nicht-Substantive ausgeschlossen, 4.) lemmatisiert wird und 5.) der normalisierte Text an die Liste unserer neuen Text gehängt wird.*

*Mit ```nlp()``` können wir dank des geladenen Taggers den Text bearbeiten und das Ergebnis in eine Variable ```text_nlp``` schreiben. ```spacy``` macht dabei alle Analysen auf einmal und schreibt die Ergebnisse wie etwa zu den Named Entities oder den POS-Tags in Attribute von ```text_nlp``` (```text_nlp.ents```, ```text_nlp.tag_```).*

*An die Entities kommen wir, indem wir per list comprehension - Schleife eine Liste erstellen. Der Code liest sich folgendermaßen: Für jedes Element (```w```) in ```text.nlp.ents``` schreibe mir dieses Element als string (```str()```) in eine Liste (```[]``` außenherum), aber nur, wenn das Element eine Länge größer 1 hat. Letzteres, weil ```spacy``` hier teils auch leere Elemente hat und das zu Fehlermeldungen führt.*

*Dann gehen wir die Strings in der Liste durch (```for entity in entities```) und schließen für jeden per Regulärem Ausdruck diesen aus ```text``` aus. Die Funktion ```re.sub()``` nimmt zunächst ein Muster ```entity```, welches hier noch "escapet" werden muss (```re.escape()```), weil forward slashes u.a. sonst nicht als Zeichen, sondern selbst als spezieller Regular Expression Code verstanden werden. Dann folgt das anstelle einzusetzende Muster, also in diesem Fall nichts (```''```). Angewendet werden soll es auf den Text, der in dieser Iteration gerade an der Reihe ist.*

*Da wir ```text``` nun verändert haben, stimmen die Tags in ```text_nlp``` nicht mehr damit überein, weshalb wir ```nlp()``` noch einmal laufen lassen müssen.*

*Nun schreiben wir ähnlich wie für die Entitäten unsere Tokens (die Worte), die Lemmas und die POS-Tags in Listen.*

*```text``` können wir nun aus diesen neu zusammensetzen. Erst einmal schließen wir die Nicht-Substantive aus. Es liest sich folgendermaßen: "Stelle beide Listen ```tokens``` und ```pos_tags``` nebeneinander als Tupel (```zip()```), so dass wir eine Liste mit solchen Korrespondenzen bekommen (in der Form ```(wort, pos_tag)```. An 0-ter Stelle eines solchen Tupels, hier genannt ```token_tag_tuple```, ist immer das Token und an 1-ter Stelle ist immer das POS-Tag. Diese Liste ist so lang, wie der Text Wörter hat. Nun prüfe für jedes Tupel, ob das darin vorkommende POS-Tag (```token_tag_tuple[1]```) mit dem Substantiv-Kürzel ```NN``` anfängt (```startswith()```). Wenn ja, dann nimm das Wort in diesem Tupel (```token_tag_tuple[0]```) und schreibe mir das in eine Liste (```[ ]``` außenherum). Die Elemente der Liste setzt du mir bitte getrennt durch ein Leerzeichen in einen einzelnen string (```' '.join()```)."*

*Genauso funkioniert es für die Lemmas, nur dass wir hier die Lemmas und die POS-Tags zusammenführen. Wenn wir die Nicht-Substantive behalten wollten, bräuchten wir stattdessen nur die Lemma-Liste per ```join()``` zusammenzuführen.*

*Schlussendlich fügen wir den bearbeiteten Text an die Liste der neuen Texte an mit ```append()```.*

In [66]:
new_texts = []
for text in data["texts"]:
    
    # 1.) spacy macht die ganze Arbeit
    nlp.max_length = len(text) + 1000 # dient dem Arbeitsspeichermanagement
    text_nlp = nlp(text)
    
    # 2.) Herausfiltern der Named Entities in eine Liste und dann Ausschluss
    entities = [str(w) for w in text_nlp.ents if len(w) > 1]

    for entity in entities:
        text = re.sub(re.escape(entity), '', text)
        
    # Wenn Entities ausgeschlossen, nlp() erneut ausführen (text hat sich geändert)
    text_nlp = nlp(text)
    
    # Tokens, Lemmas und POS-Tags von text_nlp in Listen schreiben
    tokens = [str(token.text) for token in text_nlp]
    lemmas = [str(token.lemma_) for token in text_nlp]
    pos_tags = [str(token.tag_) for token in text_nlp]
    
    # 3.) Nicht-Substantive ausschließen
    
    text = ' '.join([token_tag_tuple[0] for token_tag_tuple in zip(tokens, pos_tags) if token_tag_tuple[1].startswith('NN')])
    
    # 4.) Lemmatisieren mit Nicht-Substantiven ausgeschlossen    
    text = ' '.join([lemma_tag_tuple[0] for lemma_tag_tuple in zip(lemmas, pos_tags) if lemma_tag_tuple[1].startswith('NN')])
    
    # Lemmatisieren ohne Ausschluss
    # text = ' '.join(lemmas)
    
    # 5.) An die Liste der neuen Texte anhängen
    new_texts.append(text)

100 / 1763
200 / 1763
300 / 1763
400 / 1763
500 / 1763
600 / 1763
700 / 1763
800 / 1763
900 / 1763
1000 / 1763
1100 / 1763
1200 / 1763
1300 / 1763
1400 / 1763
1500 / 1763
1600 / 1763
1700 / 1763


Die so erstellte Liste mit bearbeiteten Texten können wir in eine Spalte unseres DataFrames schreiben, per ```assign()```. Die Spalte sollte natürlich sinnvoll benannt sein und ausdrücken, was in etwa gemacht wurde.

In [63]:
data = data.assign(text_no_entities_nouns_lemmas = new_texts)

Den obigen Code könnten wir auch anpassen, indem wir bestimmte Blöcke auskommentieren (```#``` davorsetzen). So können wir verschiedene Fassungen der Texte erhalten, in denen z.B. nur die Entities oder nur die Substantive ausgeschlossen sind. Da es lang dauert, all das durchlaufen zu lassen, habe ich das mal vorbereitet als CSV. Mit der ```pandas```-Funktion ```read_csv``` kann man diese komfortabel als Tabelle einlesen. Oft muss man nichts weiter einstellen, hier aber schon (siehe kursiv).

*Dabei sollte man immer auch einen Separator einstellen, der bei Comma Separated Value - Files oft das Komma ist, in diesem Fall aber das Tab (```sep='\t'```). Die Zahlen des Index stehen in der Datei an 0-ter Stelle der Spalten und wir wollen sie als Index (```index_col=0```). Da sich leider in einer Zeile fehlende Werte eingeschlichen haben, müssen wir diese ausschließen (```dropna()```), wobei wir natürlich die entsprechenden Zeilen und nicht die ganzen Spalten löschen wollen (```axis=0``` statt ```1```).*

In [25]:
data = pd.read_csv("newsgroup_dataframe.csv", sep='\t', index_col=0).dropna(axis=0)

In [85]:
data

,filenames,newsgroup,texts,text_no_entities_nouns,text_nouns_lemmas,text_no_entities_nouns_lemmas,text_no_entities
0,talk.politics.mideast/76141,17,Well i'm not sure about the story nad it did s...,story nad statement u.s. media ruin israels re...,story nad statement u.s. media ruin israels re...,story nad statement u.s. media ruin israels re...,Well i'm not sure about the story nad it did s...
1,talk.politics.mideast/76350,17,Although I realize that principle is not one o...,principle point question sort country tank cha...,principle point question sort country tank cha...,principle point question sort country tank cha...,Although I realize that principle is not one o...
2,rec.sport.hockey/54242,10,"Well, I will have to change the scoring on my ...",scoring playoff pool time scoring rule tomorro...,scoring playoff pool time scoring rule tomorro...,scoring playoff pool time scoring rule tomorro...,"Well, I will have to change the scoring on my ..."
3,soc.religion.christian/20631,15,"\n \nI read somewhere, I think in Morton Smit...",jesus magician _ lazarus tomb part initiation ...,morton smith _ jesus magician _ lazarus tomb p...,jesus magician _ lazarus tomb part initiation ...,"\n \nI read somewhere, I think in _Jesus the..."
4,talk.politics.mideast/75396,17,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...,sound peace process palestinean prediction tab...,sound peace process palestinean prediction tab...,sound peace process palestinean prediction tab...,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...
5,soc.religion.christian/21397,15,"Hi Damon, No matter what system or explanatio...",damon system explanation creation premise crea...,damon system explanation creation premise crea...,damon system explanation creation premise crea...,"Hi Damon, No matter what system or explanatio..."
6,rec.sport.hockey/53733,10,"\nAnd of course, Mike Ramsey was (at one time)...",course time captain buffalo pittsburgh current...,course mike ramsey time captain buffalo pittsb...,course time captain buffalo pittsburgh current...,"\nAnd of course, was (at one time) the captai..."
7,rec.sport.hockey/53730,10,"As I promised, I would give you the name of th...",name panther president huizenga team name pres...,name panther president huizenga team name bill...,name panther president huizenga team name pres...,"As I promised, I would give you the name of th..."
8,soc.religion.christian/20601,15,The concept of God as a teacher is indeed inte...,concept god teacher curve concept father child...,concept god teacher curve concept father child...,concept god teacher curve concept father child...,The concept of God as a teacher is indeed inte...
9,rec.sport.hockey/53871,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...,game(s adirondack cdi adirondack series spring...,game(s adirondack cdi adirondack series round ...,game(s adirondack cdi adirondack series spring...,GAME(S) ---------------\nADIRONDACK 6\tCDI 2\t...


Nun haben wir alle Daten, die wir möchten. Der Rest der Verarbeitung geht genau so vonstatten, wie Sie es aus dem ersten Teil kennen. Im Teil zur LDA lernen Sie allerdings noch ein paar weitere Einstellungen kennen.

### DTMs

In [26]:
vectorizer = CountVectorizer(
        analyzer = "word",
        stop_words = "english",
        lowercase = True,
        ngram_range = ( 1 , 1 ),
        min_df = 0,
        max_df = len(data["texts"]),
        max_features = 100
        )
X = vectorizer.fit_transform(data["texts"])
wordlist_X = list(vectorizer.get_feature_names())

dataframe = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
dataframe["filenames"] = data_all.filenames
dataframe = dataframe.set_index("filenames")

In [27]:
# Umsetzung mit TfidVectorizer()

vectorizer_tfidf = TfidfVectorizer(
        analyzer = "word",
        stop_words = "english",
        lowercase = True,
        ngram_range = ( 1 , 1 ),
        min_df = 0,
        max_df = len(data_all["texts"]),
        max_features = 100, 
        sublinear_tf = True
        )
X_tfidf = vectorizer_tfidf.fit_transform(data_all["texts"])
wordlist_X_tfidf = vectorizer_tfidf.get_feature_names()

# TfidfVectorizer-Objekt in DataFrame umwandeln (einfacher zu handhaben)

dataframe_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names())
dataframe_tfidf["filenames"] = data_all.filenames
dataframe_tfidf = dataframe_tfidf.set_index("filenames")

### LDA mit mehr Parametern

Nun haben wir unsere Texte vorbereitet, wieder in DTMs zählen lassen und können wieder mit der LDA zur Tat schreiten. Wie gesagt kommen hier nun auch ein paar "neue", einstellbare Parameter hinzu. Diese helfen dabei, das Ergebnis zu verfeinern. Obwohl ```gensim``` ein Topic Modeling Paket ist, bietet es nicht alle statistischen Einstellungsmöglichkeiten. Die, die wir haben, sind aber auch schon gut und einige für Sie relevante sind mit ihren Standardwerten unten im Code zu sehen:

- **passes**: Wie oft soll der Algorithmus durch den Corpus gehen, um seine Einordnung zu verfeinern?
- **alpha**: Wie sollen die Themen verteilt sein? Alle gleichmäßig häufig oder nur wenige sehr häufig? Hier kann man detailreich für jedes der in ```num_topics``` festgelegten Themen Vorgaben per Liste machen. Relevant ist erst einmal aber nur die Einstellung ```auto```, mit der man nicht den vorgegebenen symmetrischen Alphawert nimmt, sondern diesen aus den Daten schätzen lässt (das hört sich objektiver an, ist es statistisch gesehen aber nicht notwendigerweise)
- **eta**: Hier handelt es sich um die Verteilung der Wörter (prozentual) auf die Themen. Kann ebenfalls auf ```auto``` gestellt werden
- **iterations**: Der LDA-Algorithmus hat verschiedene Phasen, in denen er verschiedene statistische Optimierungsmöglichkeiten nutzt. Die Anzahl der Iterationen legt fest, wie oft der Algorithmus mit allen Dokumenten im Corpus einen Anlauf für eine dieser Optimierungsmöglichkeiten machen soll. Das hört sich so an wie **passes**, passiert aber tatsächlich innerhalb eines einzigen **pass**.

In [30]:
dictionary_X = corpora.Dictionary([wordlist_X])
# dictionary_X_tfidf = corpora.Dictionary([wordlist_X_tfidf])

lda = LdaModel(
    corpus = Sparse2Corpus(X.transpose()),
    id2word = dictionary_X,
    num_topics = 15,
    passes = 1,
    chunksize = 2000,
    update_every = 1,
    alpha = 'symmetric',
    eta = None,
    iterations = 50
        )

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) 

### Visualisierung

In [31]:
viz = pyLDAvis.gensim.prepare(lda, Sparse2Corpus(X.transpose()), dictionary_X)

In [32]:
# pyLDAvis.show(test) # dies müssen Sie benutzen, wenn Sie die Visualisierung in einem anderen Browserfenster erscheinen lassen wollen
pyLDAvis.display(viz)

Das Ergebnis ist nun wesentlich interpretierbarer und vermutlich mit etwas Zeitaufwand über verschiedene Einstellungen verfeinerbar. Für Zwecke des Information Retrieval, z.B. Themenindizierung von wissenschaftlichen Artikeln für eine Suchfunktion, ist das schon ein Fortschritt. Ein Wissenschaftler müsste sich aber immer noch Fragen stellen, ob dieses oder ein anderes Ergebnis in Bezug auf ein theoretisches Themenkonstrukt gleich gut sind, oder ob es klar schlechtere Einteilungen gibt.

### Aufgabe 4

1. Spielen Sie mit den Einstellungen des LDA-Algorithmus und suchen Sie nach besseren Ergebnissen
2. Nutzen Sie statt den unbearbeiteten Texten aus data["texts"] auch textweise die anderen bearbeiteten Versionen (für die Spaltennamen, siehe weiter oben). Das können Sie einstellen, indem Sie den entsprechenden Spaltennamen bei der Erstellung der DTM einstellen
3. Kombinieren Sie beide Herangehensweisen


# Abschließende Aufgabe

Nun erstellen Sie Ihr eigenes Topic Model, allerdings mit den schon bekannten Daten. **Bei allen Änderungen, die Sie machen, nicht vergessen zu speichern!**

Wir importieren alle Module und Daten noch einmal neu. 

In [8]:
## Mpdule für die Interaktion mit dem Betriebssystem (Dateien einlesen)
import os

## Module für die Stringverarbeitung
import string # Funktionen für strings
import re # Regular Expressions
import nltk # Natural Language Toolkit

## Module für vereinfachte Datenverarbeitung
import pandas as pd # Flexible, abfragbare Datenstrukturen: DataFrames (tabellenartig) 
import numpy as np # Für die Arbeit mit Zahlen (inklusive Methoden für Verfahren der linearen Algebra u.a.)

## Module (bzw. Funktionen) für Machine Learning
from sklearn.feature_extraction.text import CountVectorizer # Erstellung von häufigkeitsbasierten Document-Term-Matrizen
from sklearn.feature_extraction.text import TfidfVectorizer # Erstellung von tf-idf-basierten Document-Term-Matrizen

## Module für Topic-Modelling (und anderes, textbezogenes)
from gensim import corpora
from gensim.matutils import Sparse2Corpus
from gensim.models import LsiModel
from gensim.models import LdaModel

## Visualisierung (von LDA Topic-Modellen)
import pyLDAvis
import pyLDAvis.gensim # speziell nötig für den Output eines gensim LDA-Topic-Modells

from sklearn.datasets import fetch_20newsgroups

import spacy
nlp = spacy.load('en_core_web_sm')

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

data_all = pd.DataFrame({
        "filenames" : dataset.filenames,
        "texts" : dataset.data,
        "newsgroup": dataset.target
        })
# in der nachfolgenden Zeile heckelenme durch Ihren Nutzernamen ersetzen
data_all['filenames'] = data_all['filenames'].apply(lambda x: re.sub(re.escape('/home/malte/scikit_learn_data/20news_home/20news-bydate-train/'), '', x))

Nachfolgend sehen Sie alle Kategorien und ihre entsprechenden Label. Den Code brauchen Sie nicht zwingend zu beachten, aber er wird natürlich in kursiv erklärt.

*Zunächst kopieren wir unseren ```data_all```-DataFrame und binden ihn an den Namen ```nur_zur_ansicht```, denn er ist tatsächlich nur zur Ansicht da. Dazu benutzen wir ```pandas``` ```copy```-Funktion. Wenn wir ```data_all``` direkt an ```nur_zur_ansicht``` binden würden, wäre das keine Kopie. In Python ist es so, dass ein Objekt im Arbeitsspeicher mehrere Namen haben kann. Wenn wir dann aber ```nur_zur_ansicht``` verändert hätten, hätte sich das auch auf ```data_all``` ausgewirkt! Daher lieber explizit kopieren.*

*Dann bearbeiten wir wieder die Spalte der Dateinamen, damit alles lesbarer wird. Namentlich entfernen wir die Post-IDs, so dass nur die Namen der Foren übrig bleiben. Dazu benutzen wir wieder ```apply```, ```lambda``` sowie Regular Expressions. Dies alles zu erklären, würde wie gesagt den Rahmen sprengen. Nur soviel: Die Regular Expression in ```re.search()``` bedeutet "Suche mir beliebig viele Zeichen (``` * ```), die beliebig sein können (```.```) und am Anfang stehen (```^```). Wähle Sie für ein späteres Retrieval aus (```( )```). Die Zeichenkette geht bis zum ersten Slash (```\\/```, die zwei ersten Slashes sagen dem Algorithmus, dass es sich hier nicht um einen Regular Expression Code handelt, sondern um einen normalen Slash). Danach folgen beliebig viele Zahlen ( ``` [0-9] \*``` ) bis zum Ende (```$```), aber die interessieren mich nicht.*

*Schlussendlich wählen wir die uns interessierenden Spalten aus, indem wir ihre Namen als Liste zur Indizierung benutzen. Dann schließen wir alle gedoppelten Zeilen aus (```drop_duplicates()```), da wir uns ja nur für die einzelnen Newsgroups und erstmal nicht die Texte selbst interessieren und setzen den Index neu (durch den Ausschluss kommt er leider "durcheinander"). Er wird dann als eigene Spalte eingerückt, die wir mit ```drop()``` ausschließen. Hier müssen wir lediglich die gewünschte Spalte (```index```) und die richtige Achse angeben. In DataFrames sind die Zeilen die 0-te Achse und die Spalten die 1-te.*

In [31]:
nur_zur_ansicht = data_all.copy()
nur_zur_ansicht["filenames"] = nur_zur_ansicht['filenames'].apply(lambda x: re.search('(^.*)\\/[0-9]*$', x).group(1))
nur_zur_ansicht[["filenames", "newsgroup"]].drop_duplicates().reset_index().drop('index', axis=1)

,filenames,newsgroup
0,talk.politics.mideast,17
1,alt.atheism,0
2,sci.crypt,11
3,rec.sport.hockey,10
4,soc.religion.christian,15
5,comp.sys.mac.hardware,4
6,sci.med,13
7,sci.electronics,12
8,comp.graphics,1
9,misc.forsale,6


Wählen Sie drei Newsgroups aus, die Sie gern analysieren würden und von denen Sie denken, dass sie möglicherweise gut zu kontrastieren sind. Dann bauen Sie Ihren eigenen Datensatz, wie ich es weiter oben für meine drei Gruppen schon demonstriert hatte. Hier müssen Sie nichts weiter tun, als statt 15, 17 und 10 die IDs der gewünschten Gruppen einzutragen. Sie dürfen auch mehr als drei nehmen, aber dann kann es sein, dass die Berechnungen etwas länger dauern.

In [32]:
# Bitte nicht vergessen, statt heckelenme hier Ihren Server-Nutzernamen einzutragen
data_all['filenames'] = data_all['filenames'].apply(lambda x: re.sub(re.escape('/home/malte/scikit_learn_data/20news_home/20news-bydate-train/'), '', x))
data = data_all[ data_all["newsgroup"].isin([15, 17, 10]) ]
data = data.reset_index().drop("index", axis=1)

In [34]:
# Anschauen:

data.head(10)

,filenames,texts,newsgroup
0,talk.politics.mideast/76141,Well i'm not sure about the story nad it did s...,17
1,talk.politics.mideast/76350,Although I realize that principle is not one o...,17
2,rec.sport.hockey/54242,"Well, I will have to change the scoring on my ...",10
3,soc.religion.christian/20631,"\n \nI read somewhere, I think in Morton Smit...",15
4,talk.politics.mideast/75396,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...,17
5,soc.religion.christian/21397,"Hi Damon, No matter what system or explanatio...",15
6,rec.sport.hockey/53733,"\nAnd of course, Mike Ramsey was (at one time)...",10
7,rec.sport.hockey/53730,"As I promised, I would give you the name of th...",10
8,soc.religion.christian/20601,The concept of God as a teacher is indeed inte...,15
9,rec.sport.hockey/53871,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...,10


Der nachfolgende Schritt ist freiwillig, aber möglicherweise spannend für Sie: Schließen Sie Named Entities und Nicht-Substantive aus und lemmatisieren Sie. Erklärungen zum Code finden Sie weiter oben, aber Sie müssen daran auch nichts ändern (von daher können Sie die Zelle einfach ausführen). Diesen Schritt können Sie auch erst einmal auslassen und so die LDA durchführen. Sollte Sie das Ergebnis nicht befriedigen, können Sie einen Durchlauf mit voriger Textnormalisierung starten.

**Vorsicht: Diese Schleife kann länger dauern (5-10 Minuten, je nach Textmenge). Sie ist erst fertig, wenn statt einem Stern bei ```In [ ]``` eine Zahl erscheint.**

In [ ]:
new_texts = []
for text in data["texts"]:
    
    # Named Entity Recognition und Ausschluss von Personennamen
    nlp.max_length = len(text) + 1000 # dient dem Arbeitsspeichermanagement
    text_nlp = nlp(text)

    entities = [str(w) for w in text_nlp.ents if len(w) > 1]

    for entity in entities:
        text = re.sub(re.escape(entity), '', text)
        
    # Wemm Entities ausgeschlossen, nlp() erneut ausführen
    text_nlp = nlp(text)
    
    # Tokens, Lemmas und POS-Tags von text_nlp in Listen schreiben
    tokens = [str(token.text) for token in text_nlp]
    lemmas = [str(token.lemma_) for token in text_nlp]
    pos_tags = [str(token.tag_) for token in text_nlp]
    
    # Nicht-Substantive ausschließen
    
    text = ' '.join([token_tag_tuple[0] for token_tag_tuple in zip(tokens, pos_tags) if token_tag_tuple[1].startswith('NN')])
    
    # Lemmatisieren mit Nicht-Substantiven ausgeschlossen    
    text = ' '.join([lemma_tag_tuple[0] for lemma_tag_tuple in zip(lemmas, pos_tags) if lemma_tag_tuple[1].startswith('NN')])
    
    # Lemmatisieren ohne Ausschluss
    # text = ' '.join(lemmas)
    
    new_texts.append(text)

Die normalisierten Texte in eine eigene Spalte schreiben.

In [ ]:
data = data.assign(text_no_entities_nouns_lemmas = new_texts)

In [ ]:
# Anschauen:
data.head(10)

### DTM

Nun erstellen Sie Ihre Document-Term-Matrix. Ob es eine DTM mit den Häufigkeiten oder der TF-IDF-Metrik ist, ist Ihre Entscheidung. Auch hier kommen Sie möglicherweise nach einem ersten Durchlauf wieder und wählen einen anderen Weg. Die Einstellungen können Sie wie in Teil 1 erklärt ebenfalls ändern. Wenn Sie vorher die komplexere Textnormalisierung durchgeführt haben, können Sie statt der Spalte ```data["text"]``` hier auch die Spalte ```data["text_no_entities_nouns_lemmas"]``` nehmen.

In [ ]:
vectorizer = CountVectorizer(
        analyzer = "word",
        stop_words = "english",
        lowercase = True,
        ngram_range = ( 1 , 1 ),
        min_df = 0,
        max_df = len(data["texts"]),
        max_features = 100
        )
X = vectorizer.fit_transform(data["texts"])
wordlist_X = list(vectorizer.get_feature_names())

dataframe = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
dataframe["filenames"] = data_all.filenames
dataframe = dataframe.set_index("filenames")

In [ ]:
# Umsetzung mit TfidVectorizer()

vectorizer_tfidf = TfidfVectorizer(
        analyzer = "word",
        stop_words = "english",
        lowercase = True,
        ngram_range = ( 1 , 1 ),
        min_df = 0,
        max_df = len(data_all["texts"]),
        max_features = 100, 
        sublinear_tf = True
        )
X_tfidf = vectorizer_tfidf.fit_transform(data_all["texts"])
wordlist_X_tfidf = vectorizer_tfidf.get_feature_names()

# TfidfVectorizer-Objekt in DataFrame umwandeln (einfacher zu handhaben)

dataframe_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names())
dataframe_tfidf["filenames"] = data_all.filenames
dataframe_tfidf = dataframe_tfidf.set_index("filenames")

### LDA

Jetzt die LDA. Hier können Sie mit den Einstellungen experimentieren (siehe Erklärungen in Teil 2). Sie müssen das allerdings nicht tun und können sich z.B. auch auf die Einstellungen bei der Erstellung der DTM und/oder die eventuelle, vorherige Normalisierung im Sinne NER, POS-Tagging und Co. verlassen.

Achten Sie darauf, dass Sie die richtige DTM (entweder ```X``` oder ```X_tfidf```) und den entsprechenden Dictionary wählen.

In [ ]:
dictionary_X = corpora.Dictionary([wordlist_X])
# dictionary_X_tfidf = corpora.Dictionary([wordlist_X_tfidf])

lda = LdaModel(
    corpus = Sparse2Corpus(X.transpose()),
    id2word = dictionary_X,
    num_topics = 15,
    passes = 1,
    chunksize = 2000,
    update_every = 1,
    alpha = 'symmetric',
    eta = None,
    iterations = 50
        )

### Visualisierung

Nun können Sie visualisieren. Bitte erstellen Sie einen Textblock unter diesem hier, in dem Sie einerseits erklären, welche Eintellungen Sie vorgenommen haben (bzw. welche sich als ungünstig erwiesen haben) und andererseits, wie Sie die Ergebnisse interpretieren. Ist das Ergebnis "plausibel" oder gab es andere Lösungen, die vergleichbares hervorgebracht hätten? Würden Sie damit weiterarbeiten? Wie gut beschreibt die Themenverteilung welche Aspekte der Newsgroups? Ein paar Zeilen reichen.

Einen neuen Textblock erstellen Sie mit dem Plus-Symbol in der oberen Leiste. Rechts daneben müssen Sie dann noch im Dropdown-Menü auf "Markdown" klicken.

In [ ]:
viz = pyLDAvis.gensim.prepare(lda, Sparse2Corpus(X.transpose()), dictionary_X)

In [ ]:
# pyLDAvis.show(test) # dies müssen Sie benutzen, wenn Sie die Visualisierung in einem anderen Browserfenster erscheinen lassen wollen
pyLDAvis.display(viz)